# RNN을 이용한 텍스트 생성
- '경마장에 있는 말이 뛰고 있다'
- '그의 말이 법이다'
- '가는 말이 고와야 오는 말이 곱다'

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
# 3개의 문장을 변수에 저장
text="""경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

In [3]:
# 단어 집합 생성
t = Tokenizer()
t.fit_on_texts([text])

{'가는': 8,
 '경마장에': 2,
 '고와야': 9,
 '곱다': 11,
 '그의': 6,
 '뛰고': 4,
 '말이': 1,
 '법이다': 7,
 '오는': 10,
 '있는': 3,
 '있다': 5}

In [4]:
# 단어 집합 크기 설정
vocab_size = len(t.word_index) + 1
# 케라스 토크나이저의 정수 인코딩은 인덱스가 1부터 시작하지만,
# 케라스 원-핫 인코딩에서 배열의 인덱스가 0부터 시작하기 때문에
# 배열의 크기를 실제 단어 집합의 크기보다 +1로 생성해야 함
print('단어 집합의 크기: %d' % vocab_size)

단어 집합의 크기: 12


In [6]:
sequences = []
for line in text.split('\n'): # \n을 기준으로 문장 토큰화
  encoded = t.texts_to_sequences([line])[0]
  for i in range(1, len(encoded)):
    sequence = encoded[:i+1]
    sequences.append(sequence)

print('학습에 사용할 샘플의 개수: %d' % len(sequences))

학습에 사용할 샘플의 개수: 11


In [8]:
# 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
max_len = max(len(s) for s in sequences)
print('샘플의 최대 길이: ', max_len)

샘플의 최대 길이:  6


In [9]:
# 패딩: 문장의 길이를 최대 길이에 맞추어 주는 방법 - 남는 공간에는 0을 할당한다
# 전체 샘플의 길이를 6(가장 긴 샘플의 길이)으로 패딩
# 'pre'옵션을 주면 앞을 0으로 할당한다
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
sequences[:5]

array([[0, 0, 0, 0, 2, 3],
       [0, 0, 0, 2, 3, 1],
       [0, 0, 2, 3, 1, 4],
       [0, 2, 3, 1, 4, 5],
       [0, 0, 0, 0, 6, 1]], dtype=int32)

In [10]:
# 각 샘플의 마지막 단어를 레이블로 분리
X = sequences[:, :-1]
y = sequences[:, -1]
# 리스트의 마지막 값을 제외하고 저장한 것은 X
# 리스트의 마지막 값만 저장한 것은 y. 이는 레이블에 해당된다.
X[:3]

array([[0, 0, 0, 0, 2],
       [0, 0, 0, 2, 3],
       [0, 0, 2, 3, 1]], dtype=int32)

In [11]:
# 레이블 데이터 y에 대해서 원-핫 인코딩을 수행
Y = to_categorical(y, num_classes=vocab_size)
Y[:3]

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

## 모델 정의

- Embedding
- LSTM

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [13]:
model = Sequential()

model.add(Embedding(input_dim=vocab_size, output_dim=5, input_length=max_len-1))
model.add(LSTM(32))
model.add(Dense(units=vocab_size, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 5, 5)              60        
_________________________________________________________________
lstm (LSTM)                  (None, 32)                4864      
_________________________________________________________________
dense (Dense)                (None, 12)                396       
Total params: 5,320
Trainable params: 5,320
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(
    loss='categorical_crossentropy', 
    optimizer='adam',
    metrics=['accuracy'])

In [15]:
history = model.fit(X, Y, epochs=200, verbose=0)

In [16]:
history.history['accuracy'][-1]

0.7272727489471436

## 모델 검증

In [17]:
def sentence_generation(model, t, current_word, n):
  init_word = current_word
  sentence = ''
  for _ in range(n):
    encoded = t.texts_to_sequences([current_word])[0]
    encoded = pad_sequences([encoded], maxlen=5, padding='pre')
    # result = model.predict_classes(encoded, verbose=0) # deprecate predict_classes
    result = np.argmax(model.predict(encoded), axis=-1)

    for word, index in t.word_index.items():
      if index == result:
        break
    current_word = current_word + ' ' + word
    sentence = sentence + ' ' + word

  sentence = init_word + sentence
  return sentence

## 모델 변화
- Embedding Vector 갯수: [2, 4, 6, 8]
- LSTM 노드 갯수: [24, 32]

In [25]:
def execute_model(n_embed, n_lstm):
  model = Sequential()
  
  model.add(Embedding(input_dim=vocab_size, output_dim=n_embed, input_length=max_len-1))
  model.add(LSTM(n_lstm))
  model.add(Dense(units=vocab_size, activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  history = model.fit(X, Y, epochs=200, verbose=0)
  accuracy = history.history['accuracy'][-1]

  print("Accuracy: {:.4f}".format(accuracy))

  print(sentence_generation(model, t, '경마장에', 3))
  print(sentence_generation(model, t, '그의', 2))
  print(sentence_generation(model, t, '가는', 5))

for n_embed in [2, 4, 6, 8]:
  for n_lstm in [24, 32]:
    print("===================")
    print("n_embed={}".format(n_embed), "n_lstm={}".format(n_lstm))
    execute_model(n_embed, n_lstm)
    print()

n_embed=2 n_lstm=24
Accuracy: 0.4545
경마장에 말이 말이 말이
그의 말이 말이
가는 말이 말이 말이 말이 뛰고

n_embed=2 n_lstm=32
Accuracy: 0.5455
경마장에 말이 말이 말이
그의 말이 말이
가는 말이 말이 말이 있다 곱다

n_embed=4 n_lstm=24
Accuracy: 0.5455
경마장에 말이 말이 뛰고
그의 말이 말이
가는 말이 말이 말이 뛰고 있다

n_embed=4 n_lstm=32
Accuracy: 0.6364
경마장에 말이 말이 뛰고
그의 말이 말이
가는 말이 말이 말이 말이 곱다

n_embed=6 n_lstm=24
Accuracy: 0.6364
경마장에 말이 말이 뛰고
그의 말이 말이
가는 말이 말이 말이 있다 있다

n_embed=6 n_lstm=32
Accuracy: 0.7273
경마장에 말이 말이 뛰고
그의 말이 말이
가는 말이 말이 오는 말이 곱다

n_embed=8 n_lstm=24
Accuracy: 0.7273
경마장에 말이 말이 뛰고
그의 말이 말이
가는 말이 말이 오는 곱다 곱다

n_embed=8 n_lstm=32
Accuracy: 0.6364
경마장에 말이 말이 뛰고
그의 말이 말이
가는 말이 말이 오는 말이 곱다

